## Requirements

In [1]:
from urllib.request import urlretrieve
import json

from conllu import parse

from collections import defaultdict

from string import punctuation

## Download UD data

In [3]:
urlretrieve('https://raw.githubusercontent.com/UniversalDependencies/UD_French-GSD/master/fr_gsd-ud-train.conllu','train.conllu')

('train.conllu', <http.client.HTTPMessage at 0x7f6b04307a90>)

In [4]:
with open('train.conllu', 'r') as f:
    data = f.read()
sentences = parse(data)

## Select relevant words

In [ ]:
# take words from UD (filtering some out)

sampled_vocab = defaultdict(lambda: set())
my_vocab = {}

for idx, sent in enumerate(sentences):
    
    for t in sent:
        #print(" ".join([f'{k}: {t[k]}' for k in t]))
        if ((t['misc'] and t['misc'].get('Idiom')) or # no idioms
            any(p in t['form'] for p in punctuation) or  # no words containing punctuation
            (t['feats'] and 
            (t['feats'].get('Number') not in {'Sing', None} or  # only singular
            t['feats'].get('Person') not in {'3', None} or  # only 3rd person
            t['feats'].get('Tense') not in {'Pres', None} or    # only present
            t['feats'].get('Mood') not in {'Ind', None} or  # only indicative
            t['feats'].get('VerbForm') not in {'Fin', None})    # no gerunds, infinitives, etc)
            )):
            continue

        if t['feats'] and t['feats'].get('Gender'):
            sampled_vocab[t['upos'] + '_' + t['feats']['Gender']].add(t['form'].lower())
        else:
            sampled_vocab[t['upos']].add(t['form'].lower())


for k in sampled_vocab:
    print(k + "\t\t" + str(sampled_vocab[k]))

ADV		{'pacifiquement', 'comment', 'bénévolement', 'expérimentalement', 'relativement', 'autrement', 'vainement', 'parallèlement', 'approximativement', 'notammanent', 'authentiquement', 'assurément', 'guère', 'préalablement', 'simplement', 'indépendamment', 'physiquement', 'jadis', 'candidement', 'dorénavant', 'près', 'étroitement', 'accidentellement', 'énormément', 'mêmes', 'gratuitement', 'naturellement', 'viscéralement', 'subsidiairement', 'soit', 'transitivement', 'juste', 'journellement', 'considérablement', 'usque', 'gravement', 'tout', 'avec', 'conjointement', 'petit', 'activement', 'évidemment', 'pourtant', 'grossièrement', 'nécessairement', 'ci', 'essentiellement', 'joliment', 'culturellement', 'commercialement', 'durant', 'quelques', 'toutefois', 'clairement', 'verbalement', 'hors', 'thermodynamiquement', 'franchement', 'successivement', 'dessous', 'allégrement', 'parfaitement', 'pseudo', 'seulement', 'dramaturgiquement', 'courageusement', 'phonétiquement', 'absolument', 'inst

In [ ]:
# creating my vocabulary

my_vocab['VERB'] = sampled_vocab['VERB']

my_vocab['det_fem'] = sampled_vocab['DET_Fem']
my_vocab['det_masc'] = sampled_vocab['DET_Masc']
my_vocab['det_epic'] = sampled_vocab['DET']

other_epicenes = sampled_vocab['NOUN_Fem'].intersection(sampled_vocab['NOUN_Masc']) # nouns present in both sets are considered epicene
all_epicenes = sampled_vocab['NOUN'].union(other_epicenes)
my_vocab['NOUN_Epic'] = all_epicenes
my_vocab['NOUN_Fem'] = sampled_vocab['NOUN_Fem'].difference(all_epicenes) # remove epicenes
my_vocab['NOUN_Masc'] = sampled_vocab['NOUN_Masc'].difference(all_epicenes)   # remove epicenes

other_epicenes = sampled_vocab['ADJ_Fem'].intersection(sampled_vocab['ADJ_Masc']) # adjectives present in both sets are considered epicene
all_epicenes = sampled_vocab['ADJ'].union(other_epicenes)
my_vocab['adj_fem'] = sampled_vocab['ADJ_Fem'].difference(all_epicenes) # remove epicenes
my_vocab['adj_masc'] = sampled_vocab['ADJ_Masc'].difference(all_epicenes) # remove epicenes
my_vocab['adj_epic'] = all_epicenes

my_vocab['Prep'] = sampled_vocab['ADP']

for k in my_vocab:
    print(k)
    print(str(len(my_vocab[k])) + "\t" + str(my_vocab[k]))
    print()

VERB
1026	{'scandalise', 'conclut', 'réside', 'comporte', 'dévoile', 'groupe', 'recoupe', 'viennent', 'épouse', 'voyage', 'anime', 'concède', 'révolutionne', 'éclate', 'fournit', 'cite', 'déplace', 'prive', 'enregistre', 'persiste', 'convoque', 'repart', 'lutte', 'dejeune', 'vaut', 'sculpte', 'affecte', 'observe', 'fuit', 'évanouit', 'postule', 'investit', 'poursuit', 'examine', 'dit', 'confie', 'amplifie', 'souvient', 'voici', 'embrasse', 'chauffe', 'résilie', 'concourt', 'apparie', 'voit', 'tue', 'apparait', 'referme', 'croise', 'suppose', 'coulisse', 'aspire', 'profite', 'excuse', 'vise', 'contrarie', 'collectionne', 'permet', 'crève', 'sort', 'moque', 'tient', 'préside', 'limite', 'renonce', 'aggrave', 'égalise', 'contrôle', 'gère', 'éternise', 'avertit', 'fascine', 'tousse', 'combat', 'fume', 'remet', 'recrute', 'renseigne', 'meurt', 'valorise', 'interroge', 'importe', 'garantit', 'émet', 'crée', 'explore', 'apprécie', 'autorise', 'livre', 'quitte', 'bâtit', 'élance', 'réitère', '

In [ ]:
# getting rid of what I don't want

my_vocab['det_fem'] = {'telle', 'cette', 'toute', 'la', 'une', 'ladite', 'sa', 'ma', 'ta'}
my_vocab['det_masc'] = {'tel', 'ce', 'tout', 'le', 'un', 'ledit', 'son', 'mon', 'ton'}
my_vocab['det_epic'] = {'quelque', 'votre', 'notre', 'chaque', 'leur'}


my_vocab['Prep'] = {'hors', 'contre', 'avant', 'pas', 'en', 'selon', 'après', 'alias', 'plus', 'à', 'jusque', 'concernant', 
                    'depuis', 'par', 'avec', 'entre', 'envers', 'durant', 'sans', 'dès', 'vers', 'courant', 'derrière', 'sous', 
                    'devant', 'de', 'pour', 'suivant', 'chez', 'dans', 'comme', 'via', 'outre', 'moyennant', 'sur'}

vowels = ('a', 'e', 'i', 'o', 'u', 'y', 'à', 'é', 'è', 'ê', 'â', 'î', 'ï', 'ü', 'ë', 'h')
my_vocab['NOUN_Epic'] = {w for w in my_vocab['NOUN_Epic'] if not w.startswith(vowels)}
my_vocab['NOUN_Fem'] = {w for w in my_vocab['NOUN_Fem'] if not w.startswith(vowels)}
my_vocab['NOUN_Masc'] = {w for w in my_vocab['NOUN_Masc'] if not w.startswith(vowels)}

my_vocab['adj_epic'] = {w for w in my_vocab['adj_epic'] if not w.startswith(vowels)}
my_vocab['adj_fem'] = {w for w in my_vocab['adj_fem'] if not w.startswith(vowels)}
my_vocab['adj_masc'] = {w for w in my_vocab['adj_masc'] if not w.startswith(vowels)}

# enlever les doublons et convertir en liste
keys = ['VERB', 'adj_masc', 'NOUN_Masc', 'adj_fem', 'NOUN_Fem', 'adj_epic', 'NOUN_Epic', 'Prep', 'det_epic', 'det_masc', 'det_fem']
 
for i in range(11):
    for j in range(i+1,11):
        my_vocab[keys[i]].difference_update(my_vocab[keys[j]])
    if keys[i] in {'NOUN_Fem', 'NOUN_Masc'}: # separate gendered nouns into ambiguous context or not
        nouns_list = list(my_vocab[keys[i]])
        my_vocab[keys[i]] = nouns_list[0:1000]
        my_vocab[keys[i] + '_ctx_amb'] = nouns_list[1000:2000]
    elif keys[i] in {'adj_fem', 'adj_masc'}:    # keep the same number of feminine and masculine nouns
        my_vocab[keys[i]] = list(my_vocab[keys[i]])[:800]
    else:
        my_vocab[keys[i]] = list(my_vocab[keys[i]])

In [ ]:
for k in my_vocab:
    print(k)
    print(str(len(my_vocab[k])) + "\t" + str(my_vocab[k]))
    print()

VERB
870	['scandalise', 'conclut', 'réside', 'comporte', 'dévoile', 'recoupe', 'viennent', 'épouse', 'anime', 'concède', 'révolutionne', 'éclate', 'fournit', 'cite', 'déplace', 'prive', 'enregistre', 'persiste', 'convoque', 'repart', 'dejeune', 'vaut', 'sculpte', 'affecte', 'observe', 'fuit', 'évanouit', 'postule', 'investit', 'poursuit', 'examine', 'dit', 'confie', 'amplifie', 'souvient', 'voici', 'embrasse', 'résilie', 'concourt', 'apparie', 'voit', 'tue', 'apparait', 'referme', 'croise', 'suppose', 'coulisse', 'aspire', 'profite', 'excuse', 'vise', 'contrarie', 'collectionne', 'permet', 'crève', 'moque', 'tient', 'préside', 'renonce', 'aggrave', 'égalise', 'gère', 'éternise', 'avertit', 'fascine', 'tousse', 'fume', 'remet', 'recrute', 'renseigne', 'meurt', 'valorise', 'interroge', 'importe', 'garantit', 'émet', 'crée', 'explore', 'apprécie', 'autorise', 'quitte', 'bâtit', 'élance', 'réitère', 'blesse', 'tarde', 'signifie', 'prolonge', 'contourne', 'réifie', 'soutient', 'inverse', 'e

## Save vocabulary to json

In [ ]:
with open("full_vocabulary.json", "w") as outfile:
    json.dump(my_vocab, outfile)

## Add coordinating conjunctions and personal pronouns to the vocabulary